# MacroServer and its elements
<br>
<br>
<img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" />
Sardana-Training by ALBA Synchrotron is licensed under the Creative Commons Attribution 4.0 International License.  
To view a copy of this license, visit http://creativecommons.org/licenses/by/4.0/.

## Contents

* MacroServer overview
* Macro features
* Door - entry point to the MacroServer
* Generic Scan Framework
* Recorders
* Spock CLI - single point of control 
* Macro execution widgets

## [MacroServer overview](http://www.sardana-controls.org/en/latest/devel/overview/overview_macroserver.html)

* Is a controlled environment to run procedures, called macros, sequentially or simultaneously
* Provides a standard catalogue of procedures both user and expert
* Allows to plug in new/custom procedures written as Python functions or classes (advanced)
* Connects to device Pool(s) and allows to act on their elements from within macros

## Macro features

[How to write macros](http://www.sardana-controls.org/en/latest/devel/howto_macros/macros_general.html)

### Macro parameters

* Benefits
 * The parameters validation prevents runtime errors
 * Spock offers autocompletion facilities
 * Macro execution widgets offers list of allowed parameter values in combo boxes
 * Autogenerated documentation

### Macro parameters

* Parameters are defined either as asrgument of the `macro` decorator for the macro function or as `param_def` class member for the macro class
* A parameter is characterized by: name, type, default value and description
* Parameter values arrives as positional arguments to the macro function or to the run method of the macro class
* Optional parameters would allow to postpone the parameter input/assignment until runtime - see [#285](https://github.com/sardana-org/sardana/issues/285)
* 52 parameter types are defined in MacroServer e.g. Float, String, Motor, MeasurementGroup, etc.

### Macro parameters - repeat parameters

* Allows to pass as parameter value a list of repetitions of the repeat parameter member(s) - an empty list is also allowed, meaning no values
* Repeat parameters allow to:
 * restrict the minimum and/or maximum number of repetitions
 * nest repeat parameters inside of another repeat parameters
 * define multiple repeat parameters in the same macro


### Macro results

* Allows to pass the macro result to the clients (string representation) and to the wrapper macro
* Results are defined as the `result_def` class member of the macro class
* A result is characterized by: name, type, default value and description - exactly the same as macro parameter
* Multiple results are possible

### Logging in macros

* Macros may log messages to various destinations
 * Door's stream attributes - Spock subscribes to their events and prints their messages in the console
 * MacroServer log files
 * Macro logging to a file - see TODO
* Different log levels are possible e.g. output, error, debug
* Block output allows to emulate a "live" console - dynamically updated lines, tables e.g. `umv`, `ipap_homing`.

### Macro data

* Allows to pass the macro data to the clients (serialized with `pickle`) and to the wrapper macro
* Data format is totally opened
* Preserving macro data may be memory consuming. This can be disabled with the `PreserveMacroData` environmnet variable (default value is `True`)
* Scans and `ct` macro data formats should be unified - see [#500](https://github.com/sardana-org/sardana/issues/500)


### Hooks

* Python code that will be executed at a given point of the macro
* Can be a Python callable or a macro
* Hooks can be attached to the hook places that defines macros
* Macro must inherit from `Hookable` class to allow hook places
* Hook places are defined using the `hints` class member (dictionary) and the `allowsHooks` key
* Exists two possibilities to attach hooks to the macros:
 * programatically - using the `hooks` macro attribute (it requires defining a wrapper macro)
 * graphically - using the `sequencer` widget
* General hooks would allow to define hooks by means of configuration e.g. environment variables - see [#200](https://github.com/sardana-org/sardana/issues/200) and [#202](https://github.com/sardana-org/sardana/issues/202)


### Nesting macros (a.k.a. wrapper macros)

* Macros can call other macros, and these other macros can call another macros, and so on...
* Many different ways exists to execute other macros from within a macro:
 * each macro is available as a macro class method e.g. `ascan`, `lsm`
 * `createMacro` and `runMacro` (create calls the macro's `prepare` method and run calls the macro's `run` method) - may be useful if we want to manipulate the macro object e.g. attach hooks
 * `execMacro` - `prepare` and `run` called at once

### Nesting macros (a.k.a. wrapper macros)

* Many different ways exists to pass macro parameters when executing a nested macro, for example:
 * positional arguments e.g. `self.execMacro('ascan', 'mot01', '0', '100', '10', '0.2')`
 * space separated string e.g. `self.execMacro('ascan mot01 0 100 10 0.2')`
 * using object e.g. `self.execMacro('ascan', motor, 0, 100, 10, 0.2)`
* wrapper macro can be used as a sequence - e.g. multiple `execMacro` calls

### Interactive macros

* Macros can ask for user input
* Macro must be declared as interactive e.g. `imacro` decorator or `interactive` class member set to `True`
* The user input request may be characterized with: type, default value, title, unit, label, range, ...
* Spock may handle user input request either in the CLI or GUI mode - see `SPOCK_INPUT_HANDLER` in the `sardana.sardanacustomsettings`
* Interactive macros have some known issues e.g. the macro execution widgets does not support them - see [#365](https://github.com/sardana-org/sardana/issues/365)

### Ploting in macros

* Macro send request to plot to the client e.g. spock
* Uses `matplotlib.pyplot` framework API e.g. `pyplot.plot`
* Macro plots in spock do not appear until the macro finishes - see [#166](https://github.com/sardana-org/sardana/issues/166)

### Interrupting macro execution

* Macros  can be stopped, aborted or paused
* As soon as the user requested the macro interruption the macro will get interrupted on the next call of `checkPoint` (or `pausePoint`) or any of the macro API method (Macro class methods decorated with `mAPI`)
* Long sleeps should be interleaved with check points
* Macro may define `on_stop`, `on_abort` and `on_pause` method that will be called on the corresponding interruption
* Macro reserved object, for example requested with the `getMotion` method (`getMotor` will not reserve the motor object!), are stopped/aborted on the corresponding interruption (before calling the `on_stop`/`on_abort` method)

### Reporting macro progress

* Macros may report their progress while are executed
* The progress is usually expressed in percentage 0 - 100 %
* The progress update is done using the `yield` statement within the `run` method

### Macro view options

* More granual way of parametrizing macros than the environment variables
* Foreseen for changing macro's output e.g. formatting
* Implemented using `ViewOptions` environment variable
* Exists macros to manipulate the view options e.g. `setvo`, `lsvo`
* Example of usage: `wm`, `wa`

## Door

* Entry point to the macroserver
* Agregates one `MacroExecutor` object - the one responsible for handling the macro execution
* Allows to execute only one macro at the time
* Macro execution is requested with a `XML` structure or a flat list of strings representation of a macro and its parameter values
* Sequence execution is requested with a `XML` structure of a sequence of macros, its parameters values and its hook macros

## Generic Scan Framework

* Scanning modes: step, continuous (pure software - c, hardware and software - ct), hybrid
* Only the ct-mode continuous scans use events to transfer data, the rest of them read the channel values
* Provides the catalogue of turn-key scan macros e.g. `ascan`, `a2scanc`, `ascanct`, `mesh`, `timescan`, etc.
* Provides the base classes for developing custom scans e.g. `GScan`, `SScan`, `CScan`, etc.
* Custom scans are developed by instantiating the base class and developing a custom generator
* Scans are highly configurable with the [environment variables](http://www.sardana-controls.org/en/latest/users/scan.html#configuration) e.g. `ActiveMntGrp`, `ScanFile`, `ApplyInterpolation`
* Scans macros [support hooks](http://www.sardana-controls.org/en/latest/devel/howto_macros/scan_framework.html#hooks-support-in-scans) e.g. `pre-scan`, `post-acq`

## [Recorders](http://www.sardana-controls.org/en/latest/devel/howto_recorders.html#writing-recorders)

* In charge of writing data to its destinations, for example a file, the output stream or to plot it on a graph.
* TODO: draw activity diagram
* Types of recorders
 * File recorders e.g. H5, Spec, FIO
 * Output recorders e.g. json, output
 * Shared memory recorders
* File recorders are assigned by:
 * file extension, the file extension to the file recorder map is created dynamically on the MacroServer startup and may be overridden with the `SCAN_RECORDER_MAP` in the `sardana.sardanacustomsettings` module
 * explicitly by the `ScanRecorder` environment variable
* The output recorder may work in block mode - the new record will override the previous one instead of appending the new one. This is configurable with the `OutputBlock`

## [Spock](http://www.sardana-controls.org/en/latest/users/spock.html)

* CLI based on IPython
* Single point of control - allows to execute macros and control elements
* Allows to work with multiple profiles i.e. connecting to different doors
* Exposes all macros as magic commands
* Macro parameters syntax is basically a space separated string. Missing arguments are substituted by the parameter's default values (only the last ones)
* `Ctrl+C` stopps the currently running macro, the second `Ctrl+C` aborts it
* Exposes other magic commands: `expconf` \*, `showscan`, `debug`, `spsplot`, `www`, `macrodata`, `post_mortem` and `edmac` 
* Refactoring of the different IPython versions compatibility layer is necessary (several issues with IPython >= 5)

\* Expconf is executed in a separate process

## [Spock](http://www.sardana-controls.org/en/latest/users/spock.html) - repeat parameters syntax

* Support to the macros using a complex configuration of the repeat parameters was introduced recently
* Square brackets (`[` and `]`) are the grouping characters
* Each repeat parameter value must be enclosed withing a pair of square brackets
* If the repeat parameter has more than one member then each repetition also must be enclosed within a pair of square brackets
* Usage of the square brackets is optional if passing the parameter values as a space separated strings does not introduce ambiguity (only one repeat parameter and located at the end)

## Macro related widgets

* MacroExecutor: favourites macros, history of execution
* Sequencer: allows to attach macros as hooks, export/import to/from a file, import from a file with the Spock syntax
* MacroButton - a compact way of executing macros
* SardanaEditor: create a new plugin or edit an existing one, autoapply on save
* Door logging widgets - macros' output, etc.

### Other MacroServer features

* Report - write to the file the user messages (one file per day)